In [1]:
import os
os.environ['DASHSCOPE_API_KEY'] = 'sk-a31a563bce7146bf8c8ab93ad5fea537'
os.environ['COHERE_API_KEY'] = '5HPQX7dn7kV61LBb3EsQcfiqjtGbAPvk08bA4VkD'
os.environ['TAVILY_API_KEY'] = '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'

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

embd = HuggingFaceBgeEmbeddings(
    model_name='./embeddings/bge-base-zh-v1.5',
    model_kwargs={'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': True}
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=20
)
doc_splits = text_splitter.split_documents(documents=docs_list)

vectorestore = Chroma.from_documents(
    documents=doc_splits,
    collection_name='rag-chroma',
    embedding=embd
)
retriever = vectorestore.as_retriever()

No sentence-transformers model found with name ./embeddings/bge-base-zh-v1.5. Creating a new one with mean pooling.


In [15]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    datasource: Literal['vectorestore', 'web_search'] = Field(
        ...,
        description="给定用户问题，选择将其路由到网络搜索(web_search)或向量存储库(vectorstore)。"
    )

llm = ChatTongyi()
structured_llm_router = llm.with_structured_output(RouteQuery)

system = """你是用户问题 路由到 向量存储库或网络搜索 的专家。
向量存储库包含与代理、提示工程和对抗攻击相关的文档。
对于这些主题的问题，请使用向量存储库(vectorstore)。否则，请使用网络搜索(web_search)。
"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}")
    ]
)
question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "熊队会在 NFL 选秀中首轮选中哪位球员？"}
    )
)
print(question_router.invoke({"question": "代理内存的类型有哪些？"}))


datasource='web_search'
datasource='vectorestore'


In [54]:
class GradeDocuments(BaseModel):
    """用于检查检索文档相关性的二进制评分类。"""
    binary_score: str = Field(
        description="检索文档与问题的相关性，yes 表示相关，no 表示不相关"
    )

llm = ChatTongyi()
structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = '''
    你是一名评估员，需要评估检索到的文档与用户问题的相关性。
    如果文档包含与用户问题相关的关键词或语义意义，请将其评为相关。
    这不需要严格的测试，目标是过滤掉错误的检索结果。
    使用二进制评分“yes”或“no”来表示文档是否与问题相关。
'''
grade_promt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n{document}\n, User question: {question}")
    ]
)

retrieval_grader = grade_promt | structured_llm_grader

question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[2].page_content
print(doc_txt)
print(retrieval_grader.invoke({"document": doc_txt, "question": question}))


Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided by the agent.
- Inter-agent communication can trigger new natural language statements.


Retrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.
binary_score='yes'


In [21]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull('rlm/rag-prompt')

/home/zta/anaconda3/envs/langgraph/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [62]:
llm = ChatTongyi()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chian = prompt | llm | StrOutputParser()
generation = rag_chian.invoke({'context': docs, 'question': question})
print(generation)

In LLM-powered autonomous agents, memory is divided into short-term and long-term memory. Short-term memory involves in-context learning, while long-term memory allows agents to retain and recall information over extended periods using an external vector store for fast retrieval.


In [70]:
### Hallucination Grader

class GradeHallucinations(BaseModel):
    '''生成的答案与文档相关性检查的二进制评分。'''
    binary_score: str = Field(...,
        description="答案是否基于检索到的文档，'yes'或'no'。"
    )

llm = ChatTongyi()
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

system = """
你是一个评估员，负责评估大型语言模型生成的答案是否 基于/得到 一组检索到的事实的支持。
给出二进制评分“yes”或“no”。
“yes”表示答案是基于/由这些事实支持的。
"""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}")
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
print(hallucination_grader.invoke({"documents": format_docs(docs), "generation": generation}))

binary_score='yes'


In [71]:
### Answer Grader

class GradeAnswer(BaseModel):
    """判断回答是否与问题相匹配"""
    binary_score: str = Field(
        description="判断是否回答了问题，yes表示回答了，no表示没有回答"
    )
llm = ChatTongyi()
structured_llm_grader = llm.with_structured_output(GradeAnswer)

system = '''
你是一个评估员，负责评估答案是否解决了问题。
给出二进制评分“是”或“否”。
“是”表示答案解决了问题。
'''
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [90]:
### question rewriter

llm = ChatTongyi()

system = '''
你是一个问题重写者，负责将输入问题转化为更优化的版本，以便进行向量存储检索。
查看输入问题，并尝试推理其潜在的语义意图/含义。
你要做的工作是尝试重写用户所提出问题，将问题转换为更优化的版本。
仅返回优化后的问题版本即可，不要返回其他内容。
例如：
问题：agent memory
优化后：What is the role of memory in an agent's functioning?
'''
question = 'agent memory'
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

"What is the role of memory in an agent's functioning?"